In [1]:
!pip install chromadb
!pip install openai
!pip install tqdm
!pip install sounddevice
!pip install whisper
!pip install gtts
!pip install beautifulsoup4
!pip install tavily-python
!pip install lxml
!pip install pyttsx3
!pip install langchain_openai
!pip install langchain-community
!pip install langchain-google-genai
!pip install langsmith
!pip install PyPDF2

# Common dependencies for some of the above packages
# BeautifulSoup often works best with an external parser like lxml
# Tavily and OpenAI clients use an HTTP client like httpx
!pip install lxml
!pip install httpx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 97.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 101.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.9/131.9 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 8.9 MB/s et

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 10.2 MB/s eta 0:00:00


In [2]:
!pip install google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=41f0599e2e2ab1646d500e5bce0659477b3f7c7b869612b253f09d6dce5f9508
  Stored in directory: /root/.cache/pip/wheels/0c/47/f5/89b7e770ab2996baf8c910e7353d6391e373075a0ac213519e
Successfully built google-search-results


In [3]:
# ============================================
# STEP - 1 : Imports
# ============================================

# Standard Library Imports
import os
import json
import re

# Third-Party Library Imports
import chromadb
import openai
import tqdm
import pyttsx3
import gtts
from google.colab import drive
from bs4 import BeautifulSoup
from tavily import TavilyClient

# LangChain and related framework imports
import langchain
from langchain import hub
from langchain.tools import Tool
from langchain.agents import AgentExecutor, create_react_agent
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import WebBaseLoader
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser # Corrected import path
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI

In [4]:
# =========================================
# STEP - 2 : Load API keys (Colab + Env)
# =========================================
import os

# Attempt to fetch from Colab userdata if available
try:
    from google.colab import userdata
    _colab_available = True
except ImportError:
    _colab_available = False

def get_key(key_name: str) -> str:
    """Fetch API key from Colab userdata first, then environment variables."""
    key = None
    if _colab_available:
        key = userdata.get(key_name)
    if not key:
        key = os.environ.get(key_name)
    if not key:
        print(f"❌ {key_name} not found! Please add it to Colab secrets or environment variables.")
    else:
        print(f"✅ {key_name} loaded successfully!")
    return key

# Fetch all keys
OPENAI_API_KEY = get_key("OPENAI_API_KEY")
TAVILY_API_KEY = get_key("TAVILY_API_KEY")
SERPAPI_API_KEY = get_key("SERPAPI_API_KEY")
LANGCHAIN_API_KEY = get_key("LANGCHAIN_API_KEY")
# LANGSMITH_API_KEY = get_key("LANGSMITH_API_KEY")
# HF_TOKEN = get_key("HF_TOKEN")
# Optional: Pinecone if you use it
# PINECONE_API_KEY = get_key("PINECONE_API_KEY")

# Set OpenAI key for SDKs
if OPENAI_API_KEY:
    import openai
    openai.api_key = OPENAI_API_KEY


✅ OPENAI_API_KEY loaded successfully!
✅ TAVILY_API_KEY loaded successfully!
✅ SERPAPI_API_KEY loaded successfully!
✅ LANGCHAIN_API_KEY loaded successfully!


In [5]:
# ================================================
# ---------- MOUNT THE GOOGLE DRIVE -------------
# ================================================

from google.colab import drive
drive.mount('/content/drive')

print(f"✅ Google Drive mounted successfully!")


Mounted at /content/drive
✅ Google Drive mounted successfully!


In [6]:
# ============================================
# STEP - 3 : Importing all embedded chunks
# ============================================

all_embedded_file_path = "/content/drive/MyDrive/Ironhack_final_project/all_embedded_chunks.json"

with open(all_embedded_file_path, "r", encoding="utf-8") as f:
    all_embedded_chunks = json.load(f)

print(f"✅ Loaded {len(all_embedded_chunks)} chunks")
print(all_embedded_chunks[0])  # preview first chunk

✅ Loaded 164 chunks
{'video_id': 'tQ84XYcP-nA', 'chunk_index': 0, 'text': "Every week there's a new AI tool making headlines, and right now there are more AI video generators than ever. But most of them don't work as well as you'd expect. Some generate great videos, but only if you stay within their style limits. If you try to get more creative, like detailed anime or wild fantasy worlds, they often mess up, and most of the time they're too expensive or just confusing to use. I've tested every major tool that's come out recently, and I've seen where they shine and where they completely fall apart. So, in this video, I'm going to show you the AI video tools that actually deliver the kind of quality you'd want to publish. The best way to use them without wasting hours learning clunky software, and the one platform that ties everything together, so you can create full videos without switching between five different sites. Let's break it down. All right, let's start with Seedance 1.0. Sean

In [7]:
# ============================================
# STEP - 4 : RAG Pipeline with Memory
# ============================================

import os
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory

# --------------------------
# 1️ Initialize OpenAI client
# --------------------------
api_key = userdata.get("OPENAI_API_KEY")  # Load API key from Colab secrets
# client = OpenAI(api_key=api_key) # No need to re-initialize client here


# --------------------------
# 2️ Initialize OpenAIEmbeddings
# --------------------------
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=api_key
)

# --------------------------
# 3️ Create/Load vector store
# --------------------------

persist_dir = "/content/drive/MyDrive/Ironhack_final_project/chromadb_store" # file path to the vector database

vectorstore = Chroma(
    persist_directory=persist_dir,  # folder where your vector DB is stored
    embedding_function=embeddings
)


# ----------------------------------------------------------
# 4️ Add embedded chunked data to Chromadb_store (vector DB or store)
# ----------------------------------------------------------
texts = [chunk["text"] for chunk in all_embedded_chunks]
metadatas = [
    {
        "source": chunk.get("video_id") or chunk.get("article_id"),
        "chunk_index": chunk["chunk_index"]
    }
    for chunk in all_embedded_chunks
]

# Add documents with automatic embedding generation
vectorstore.add_texts(texts=texts, metadatas=metadatas)

print(f"🎉 Done! Chroma vector store is loaded")


# --------------------------
# 5️ Create retriever
# --------------------------
retriever = vectorstore.as_retriever(search_kwargs={"k": 8})  # higher k for better results


# --------------------------
# 6️ LLMChain prompt and chain
# --------------------------

prompt = PromptTemplate(
    input_variables=["context", "question", "chat_history"],
    template="""
You are a helpful AI assistant for content creators.
Always answer concisely, clearly, and in a structured format.
Prefer bullet points or numbered steps (3–6 items).
Each bullet should be 1–2 sentences max.
Do not repeat information.
If the context is incomplete, use conversation history or your own knowledge.

Conversation history:
{chat_history}

Context:
{context}

Question: {question}

Answer (concise and structured):
"""
)


# prompt = PromptTemplate(
#     input_variables=["context", "question", "chat_history"],
#     template="""
# You are a helpful AI assistant. Use the context below and the chat history to answer the question.
# If the context is incomplete, rely on conversation history or your knowledge to give the best answer.

# Conversation history:
# {chat_history}

# Context:
# {context}

# Question: {question}

# Answer:
# """
# )

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3, openai_api_key=api_key) # Pass the api_key here


# ----------------------
# Add memory here
# ----------------------
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


qa_chain = LLMChain(llm=llm, prompt=prompt)


# ----------------------------------
# 7️ Full RAG function with memory
# ----------------------------------

def ask_rag(query: str) -> str:
    """
    Full RAG pipeline: retrieves relevant chunks, uses memory, and generates answer.
    """
    # --- Retrieve relevant chunks ---
    results = retriever.get_relevant_documents(query)

    if not results:
        return "❌ No relevant documents found."

    # --- Combine retrieved chunks ---
    context = "\n\n".join([doc.page_content for doc in results])

    print("\n📝 Retrieved context preview:\n", context[:1000])

    # --- Load memory ---
    chat_history = memory.load_memory_variables({}).get("chat_history", [])

    # --- Generate answer ---
    answer = qa_chain.run({
        "context": context,
        "question": query,
        "chat_history": chat_history
    })

    # --- Save interaction into memory ---
    memory.save_context({"input": query}, {"output": answer})

    return answer

/tmp/ipython-input-1257009309.py:24: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(
/tmp/ipython-input-1257009309.py:35: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


🎉 Done! Chroma vector store is loaded


/tmp/ipython-input-1257009309.py:116: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
/tmp/ipython-input-1257009309.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  qa_chain = LLMChain(llm=llm, prompt=prompt)


In [8]:

# ✅ Run a memory test
print("\n--- Memory Test ---\n")

# First interaction
response1 = ask_rag("My name is Abhi.")
print("User: My name is Abhi.")
print("Bot:", response1)

# Second interaction (checks memory)
response2 = ask_rag("What is my name?")
print("\nUser: What is my name?")
print("Bot:", response2)

# Third interaction (longer context test)
response3 = ask_rag("Remember that I like working with RAG pipelines.")
print("\nUser: Remember that I like working with RAG pipelines.")
print("Bot:", response3)

response4 = ask_rag("What do I like working with?")
print("\nUser: What do I like working with?")
print("Bot:", response4)



--- Memory Test ---



/tmp/ipython-input-1257009309.py:131: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)



📝 Retrieved context preview:
 AI to do the work for you thank you so much for watching the video and I'll see you in next one

AI to do the work for you thank you so much for watching the video and I'll see you in next one

AI to do the work for you thank you so much for watching the video and I'll see you in next one

AI to do the work for you thank you so much for watching the video and I'll see you in next one

AI to do the work for you thank you so much for watching the video and I'll see you in next one

AI to do the work for you thank you so much for watching the video and I'll see you in next one

AI to do the work for you thank you so much for watching the video and I'll see you in next one

AI to do the work for you thank you so much for watching the video and I'll see you in next one


/tmp/ipython-input-1257009309.py:145: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa_chain.run({


User: My name is Abhi.
Bot: - Nice to meet you, Abhi!
- How can I assist you today?
- Feel free to ask any questions or request help with your content creation.

📝 Retrieved context preview:
 is a great option. Start with a prompt (like “Short-form social media video about financial literacy,” and watch as your video comes to life. You can then make changes to the video’s language, content, and more through additional text prompts. Create bite-sized video content in minutes, perfect for small teams who can’t rely on video production. Source: Invideo Use Invideo for: However, it’s not best for: Canva is a well-known online graphics editing app that has, in recent years, expanded to cover document design, presentations, mini websites, and more. Like other apps, Canva has added a slew of AI-powered content creation features over the last year, which they call “Magic Design.” Upload any existing media and Magic Design will give you matching templates, like social media templates, quote gra

In [9]:
# =============================================================================
# STEP - 5 :  Testing my RAG retrieval and its memory with multiple questions
# =============================================================================

# First question: sets some context
print(ask_rag("What is an AI image generator?"))

# Second question: relies on memory
print(ask_rag("And which one is considered the best?"))

# Third question: continues from memory
print(ask_rag("Why is it considered the best?"))


# # -----------------------
# # 8️⃣ Testing my RAG
# # -----------------------
# if __name__ == "__main__":
#     user_query = input("❓ Enter your question: ")
#     answer = ask_rag(user_query)
#     print("\n💡 Answer:\n", answer)


# # -----------------------------------------------------------
# # 8️⃣ Testing RAG with memory with continuous questioning
# # -----------------------------------------------------------

# if __name__ == "__main__":
#     print("📝 You can ask multiple questions. Type 'exit' to quit.\n")

#     while True:
#         user_query = input("❓ Enter your question: ")
#         if user_query.lower() in ["exit", "quit"]:
#             print("👋 Exiting chat. Goodbye!")
#             break

#         answer = ask_rag(user_query)
#         print("\n💡 Answer:\n", answer)
#         print("-" * 50)  # separator for readability




📝 Retrieved context preview:
 AI has revolutionized image creation. You can now type in a text prompt, like design a storefront for a cookie store, and get captivating visuals back. There are lots of different AI image generators out there, both free and also paid, and you've probably used some yourself. In this video, in partnership with Zapier, we'll look at the top seven AI image generators from Dalle 3 to Midjourney and more. Let's dive in. First up, we have Dalle 3, which is developed by OpenAI, the maker of ChatGPT. You can access this directly from chatgpt.com with a free account, although you only get two images per day. With a plus account, on the other hand, you can create more images, but plans start at $20 per month. As an alternative, you can go to Microsoft's Bing Image Creator, which allows you to create any number of images for free, and it uses Dalle 3 on the backend, the same as what you get with ChatGPT. So, what do I like or dislike about it? Well, overall, you get

**==================>>> T O O L S <<<========================**

In [10]:
# ============================================
# STEP - 6 : Retriever tool (Chroma retriever) with metadata
# ============================================

from langchain.tools import Tool

retriever_tool = Tool(
    name="Chroma Retriever",
    func=vectorstore.as_retriever().get_relevant_documents,
    description=(
        "This is the PRIMARY tool to use first. "
        "Always try this before using any other tool. "
        "It retrieves relevant document chunks from the Chroma database "
        "to answer questions when the context exists."
    )
)

print("✅ Retriever tool is loaded")

✅ Retriever tool is loaded


In [11]:
# ============================================
# STEP - 7 : Google Search tool (Tavily)
# ============================================


# from langchain.tools import Tool


tavily_api_key = userdata.get('TAVILY_API_KEY')
if not tavily_api_key:
    raise ValueError("❌ No Tavily API key found! Please add it in Colab secrets.")

from tavily import TavilyClient
tavily = TavilyClient(api_key=tavily_api_key)

def search_tavily(query: str):
    """Perform a web search using Tavily and return top results as a list."""
    results = tavily.search(query, max_results=3)
    # Return a list of result content
    return [r["content"] for r in results["results"]]

search_tool = Tool(
    name="Google Search (Tavily)",
    func=search_tavily,
    description="Use this when the question cannot be answered from the context. Returns top 3 web search results."
)

print("✅ Search tool is loaded")

# ------------------------------------
# Testing the search tool (Tavily)
# ------------------------------------

results = search_tavily("AI image generators")
print(f"🔍 Tavily Search Results:")
for i, r in enumerate(results, 1):
    print(f" {i}. {r}")

✅ Search tool is loaded
🔍 Tavily Search Results:
 1. DeepAI's Free Online AI Image Generator gives you the power to visualize your imagination in seconds. Just describe your vision and watch it come to life.
 2. * Canva’s AI image generators are available with limited use on Free accounts. * Canva’s AI image generator tools make it easy to turn text into visuals. Please be mindful that Canva doesn’t guarantee that the AI-generated images, designs, and text you generate are cleared for use (particularly if the image or design you create looks like someone else’s work). Between you and Canva, you own the designs you generate with our AI image generators, which is subject to you following our Terms. We’ve put layers of safety measures in place so you can use Magic Media's Text to Image tool, our free AI image generator from text, safely and responsibly.
 3. ## ImagineArt AI Generated Images ## All The Features That You Need In An AI Image Generator Tool ## What Makes Us The Best AI Image 

In [12]:
# ========================================================
# STEP 8 - W H I S P E R  T O O L - for Speech-to-Text
# ========================================================

# import whisper # Removed this import
# import sounddevice as sd # Keep this import if local recording is desired, but address PortAudio issue separately
from scipy.io.wavfile import write
from langchain.tools import Tool
from openai import OpenAI # Ensure OpenAI client is imported

# Load Whisper model once # Removed this line

client = OpenAI(api_key=OPENAI_API_KEY)


def record_audio(filename="input.wav", duration=5, fs=16000):
    print(f"🎤 Recording for {duration} seconds...")
    audio = sd.rec(int(duration * fs), samplerate=fs, channels=1)
    sd.wait()
    write(filename, fs, audio)
    print("✅ Recording finished")
    return filename

def transcribe_audio_openai(audio_file_path):
    """Transcribes audio using OpenAI's Whisper model."""
    with open(audio_file_path, "rb") as audio_file:
        # Use the existing 'client' object from cell oz4C828MYuR5
        # Make sure the client is initialized in a preceding cell
        transcription = client.audio.transcriptions.create(
            model="whisper-1", # OpenAI's Whisper model
            file=audio_file
        )
    return transcription.text

# --- Wrap in a Tool ---
def whisper_speech_to_text(audio_file_path):
    """Transcribes a given audio file into text using OpenAI's Whisper."""
    # Note: This now expects a file path as input, not live recording due to PortAudio issue
    # If local recording is resolved, you can add logic here to record first
    if not os.path.exists(audio_file_path):
         return f"❌ Error: Audio file not found at {audio_file_path}"

    text = transcribe_audio_openai(audio_file_path)
    return text

whisper_tool = Tool(
    name="Whisper Speech-to-Text",
    func=whisper_speech_to_text,
    description=(
        "Transcribes an audio file (provide file path) into text using OpenAI's Whisper model. "
        # Removed the part about recording live audio due to environment limitations
        "Useful for converting spoken content from a file into text for analysis or response generation."
    )
)

print("✅ Whisper tool is loaded (using OpenAI)")

✅ Whisper tool is loaded (using OpenAI)


In [13]:
# =====================================================
# STEP - 9 : TTS (Text-to-Speech) tool with OpenAI
# =====================================================

from openai import OpenAI
import os


# client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def text_to_speech(text, filename="response.mp3"):
    """
    Convert agent's text response to natural speech using OpenAI TTS
    """
    with client.audio.speech.with_streaming_response.create(
        model="gpt-4o-mini-tts",   # You can also try "gpt-4o-tts" for higher quality
        voice="alloy",             # Options: alloy, verse, sage, etc.
        input=text,
    ) as response:
        response.stream_to_file(filename)

    print(f"🔊 Saved speech to {filename}")
    return filename

# Wrap text-to-speech function as a Tool
tts_tool = Tool(
    name="Text-to-Speech",
    func=text_to_speech,
    description="Converts text into natural-sounding speech using OpenAI TTS and saves as MP3."
)

print("✅ Text-to-Speech tool is loaded")

✅ Text-to-Speech tool is loaded


In [14]:
# =================================================
# STEP - 10 : N E W S  T O O L - Powered by Google
# =================================================

import os
from langchain.utilities import SerpAPIWrapper
from langchain.agents import Tool

# --------------------------
# 1️⃣ Load SerpAPI key
# --------------------------
# For Colab, you can use userdata.get() if available
try:
    from google.colab import userdata
    serpapi_api_key = userdata.get('SERPAPI_API_KEY')
except ImportError:
    serpapi_api_key = os.getenv('SERPAPI_API_KEY')

if not serpapi_api_key:
    raise ValueError("❌ No SerpAPI API key found! Please set it in environment variables or Colab secrets.")

os.environ['SERPAPI_API_KEY'] = serpapi_api_key

# --------------------------
# 2️⃣ Initialize SerpAPI wrapper
# --------------------------
search = SerpAPIWrapper()  # automatically uses SERPAPI_API_KEY

# --------------------------
# 3️⃣ Safe search function
# --------------------------
def safe_news_search(query: str) -> str:
    """
    Searches the web using SerpAPI and returns a concise summary of the top results.

    Args:
        query (str): The search query. Must not be empty.

    Returns:
        str: Concise summary of the search results or an error message if input is invalid.
    """
    if not query or query.strip() == "":
        return "❌ Cannot perform search: the query is empty."

    # Get top 5 results for brevity
    results = search.results(query)
    organic = results.get("organic_results", [])

    if not organic:
        return "No results found for your query."

    summary_lines = []
    for i, r in enumerate(organic[:5], 1):
        title = r.get("title", "No title")
        link = r.get("link", "No link")
        snippet = r.get("snippet", "")
        summary_lines.append(f"{i}. {title}\n   {snippet}\n   🔗 {link}")

    return "\n\n".join(summary_lines)

# --------------------------
# 4️⃣ Wrap as a LangChain tool
# --------------------------
news_tool = Tool(
    name="Latest AI News",
    func=safe_news_search,
    description=(
        "Use this tool to search the web for factual information, news, or updates on a specific topic. "
        "Only call it when the user asks a factual question or requests the latest news. "
        "Do NOT use it for greetings, casual conversation, or personal opinions. "
        "The input must be a valid search query; empty queries will return an error."
    )
)

print("✅ Latest AI News tool is loaded")


✅ Latest AI News tool is loaded


In [15]:
!pip install fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=78e2769df686971a6f1d7bff84a0380ab4e9b32d485c3d6e8d8a0d50fe5be1a1
  Stored in directory: /root/.cache/pip/wheels/6e/62/11/dc73d78e40a218ad52e7451f30166e94491be013a7850b5d75
Successfully built fpdf


In [16]:
# ============================================
# STEP - 11 : SAVE CHAT AS PDF TOOL
# ============================================

from fpdf import FPDF

def save_chat_as_pdf(chat_text: str, filename="chat.pdf"):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.multi_cell(0, 10, chat_text)
    pdf.output(filename)
    return f"Chat saved to {filename}"

from langchain.agents import Tool

save_pdf_tool = Tool(
    name="SaveChatPDF",
    func=save_chat_as_pdf,
    description="Saves the current chat as a PDF file."
)



In [17]:
# ============================================
# STEP - 11 : SUMMARY TOOL
# ============================================

import requests
from bs4 import BeautifulSoup

def summarize_url(url: str):
    res = requests.get(url)
    soup = BeautifulSoup(res.text, "html.parser")
    text = soup.get_text()
    # Use your LLM to summarize
    summary = llm.predict(f"Summarize this text:\n{text}")
    return summary

url_summary_tool = Tool(
    name="URLSummary",
    func=summarize_url,
    description="Reads a URL and returns a summarized version of its content."
)


In [18]:
# ============================================
# STEP - 11 : GREETING TOOL
# ============================================
import random


def greet_user(user_input: str = None) -> str:
    greetings_general = [
        "Hello! 👋 I’m your AI Content Coach. How can I assist you today?",
        "Hi there! I’m here to help you create amazing content. What would you like to work on?",
        "Hey! Ready to improve your content? Let’s get started.",
        "Hello! I can guide you through content creation, AI tools and tips. What’s first?"
    ]

    if user_input:
        user_input_lower = user_input.lower()
        if any(word in user_input_lower for word in ["morning", "afternoon", "evening"]):
            time_word = next((w for w in ["morning", "afternoon", "evening"] if w in user_input_lower), "day")
            return f"Good {time_word.capitalize()}! I’m your AI Content Coach. How can I help you today?"
        if any(word in user_input_lower for word in ["hi", "hello", "hey"]):
            return random.choice(greetings_general)

    return random.choice(greetings_general)

greeting_tool = Tool(
    name="Greeting",
    func=greet_user,
    description="Responds naturally to greetings like 'hi', 'hello', or 'good morning'."
)


print("✅ Greeting tool is loaded")

✅ Greeting tool is loaded


**=================== > > > AGENT < < < ======================**

In [19]:
# ============================================
# STEP - 12 : Conversational A G E N T
# ============================================

from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from gtts import gTTS
from IPython.display import Audio, display
import openai
# import sounddevice as sd
import numpy as np
import tempfile
import wave


# ----------------------------------------
# Integrating list of tools to the Agent
# ----------------------------------------

tools = [greeting_tool, retriever_tool, search_tool, whisper_tool, news_tool, tts_tool, save_pdf_tool, url_summary_tool]

# ------------------------------------
# 3️⃣ Initialize Agent with Memory
# ------------------------------------

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True,
    handle_parsing_errors=True
)

print("🤖 AI Content Coach is ready! Type 'exit' to quit.")

# --------------------------
# 4️⃣ Run Conversation Loop
# --------------------------

voice_enabled = False  # 🔇 Default OFF

while True:
    user_input = input("\nYou: ")
    if user_input.lower() in ["exit", "quit"]:
        print("👋 Goodbye!")
        break

    # Toggle voice manually
    if user_input.lower() == "voice on":
        voice_enabled = True
        print("🔊 Voice enabled.")
        continue
    elif user_input.lower() == "voice off":
        voice_enabled = False
        print("🔇 Voice disabled.")
        continue

    # Agent responds
    response = agent.run(user_input)
    print("\nAI:", response)

    # Play audio only if enabled
    if voice_enabled:
        text_to_speech(response)

/tmp/ipython-input-2266210974.py:26: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


🤖 AI Content Coach is ready! Type 'exit' to quit.

You: quit
👋 Goodbye!


**=================== > > > AGENT DEPLOYMENT < < < ======================**

In [20]:
!pip install PyPDF2


In [26]:

import gradio as gr
import os
import inspect, asyncio, traceback, tempfile, logging, types
import markdown

# Path for logs
LOG_FILE = "agent_gradio.log"
logging.basicConfig(filename=LOG_FILE,
                    level=logging.INFO,
                    format="%(asctime)s %(levelname)s %(message)s")

# ----------------- Globals -----------------
voice_enabled = False


def should_use_retriever(query: str) -> bool:
    """Decide if query needs external knowledge (RAG) or just memory."""
    conversational_keywords = ["my name", "what did i say", "remember", "earlier", "last time"]
    return not any(keyword in query.lower() for keyword in conversational_keywords)


# ----------------- Core RAG Agent Logic -----------------

def chat_with_agent(message, history):
    user_query = message.strip()

    # Convert chat_history (list of tuples) into OpenAI message format
    messages = [{"role": "system", "content": "You are a helpful AI assistant. "
            "Always keep responses short, concise, and informative. "
            "Limit answers to 2–5 sentences or bullet points."}]
    for user_msg, bot_msg in history:
        messages.append({"role": "user", "content": user_msg})
        messages.append({"role": "assistant", "content": bot_msg})

    # Add the latest user query
    messages.append({"role": "user", "content": user_query})

    try:
        if should_use_retriever(user_query):
            # ✅ Use RAG pipeline (retriever + memory)
            response = ask_rag(user_query)
        else:
            # ✅ Use memory only (skip retriever)
            completion = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=messages,
                max_tokens=250
            )
            response = completion.choices[0].message.content

    except Exception as e:
        response = f"⚠️ Agent Error: {str(e)}"

    return response



# ----------------- Feature Functions -----------------
def text_to_speech(text):
    """Placeholder function to simulate text-to-speech."""
    print(f"Simulating TTS for: '{text}'")
    # In a real app, you would use a library like gTTS or pyttsx3.
    # from gtts import gTTS
    # tts = gTTS(text=text, lang='en')
    # tmp = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3")
    # tts.save(tmp.name)
    # return tmp.name
    return None # Returns None to indicate no audio file is produced in this example

def download_transcript(history):
    """Placeholder function to download chat transcript."""
    path = tempfile.NamedTemporaryFile(delete=False, suffix=".txt").name
    try:
        with open(path, "w", encoding="utf-8") as f:
            for user, assistant in history:
                f.write("User: " + str(user) + "\n")
                f.write("Assistant: " + str(assistant) + "\n")
                f.write("-" * 60 + "\n")
        return path
    except Exception as e:
        logging.exception("Failed to write transcript")
        raise

# -------------------------------------------------------------------------------------


import os

def summarize_file(file):
    """Read and summarize an uploaded file (PDF or TXT)."""
    if file is None:
        return "⚠️ Please upload a file."

    try:
        text_content = ""

        # Handle .txt files
        if file.name.lower().endswith(".txt"):
            with open(file.name, "r", encoding="utf-8", errors="ignore") as f:
                text_content = f.read()

        # Handle .pdf files
        elif file.name.lower().endswith(".pdf"):
            from PyPDF2 import PdfReader
            reader = PdfReader(file.name)
            for page in reader.pages[:5]:  # Limit to first 5 pages
                text_content += page.extract_text() or ""

        else:
            return "⚠️ Unsupported file format. Please upload a PDF or TXT file."

        if not text_content.strip():
            return "⚠️ Could not extract meaningful content from the file."

        # Summarize with AI
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "Summarize the following document in 3–4 concise sentences."},
                {"role": "user", "content": text_content[:4000]}  # Truncate for efficiency
            ],
            max_tokens=250  #lower number prevents overly long completions. In this case 250 is low which is good if you want to keep concise
        )
        return completion.choices[0].message.content

    except Exception as e:
        return f"⚠️ Error summarizing file: {str(e)}"



# def summarize_file(file):
#     """Placeholder function to summarize an uploaded file."""
#     if file is None:
#         return "Please upload a file."

#     # You would add your file reading and summarization logic here.
#     return f"This is a placeholder summary of the file at: {file.name}"


# -------------------------------------------------------------------------------------


import requests
from bs4 import BeautifulSoup



# -------------------------------------------------------------------------------------


def summarize_url(url):
    """Fetch and summarize the main text content of a URL."""
    if not url or not url.startswith(("http://", "https://")):
        return "⚠️ Please enter a valid URL (must start with http:// or https://)."

    try:
        # Fetch webpage
        response = requests.get(url, timeout=10)
        response.raise_for_status()

        # Parse HTML
        soup = BeautifulSoup(response.text, "html.parser")
        paragraphs = [p.get_text(strip=True) for p in soup.find_all("p")]
        text_content = " ".join(paragraphs[:10])  # Limit to first 10 paragraphs for efficiency

        if not text_content.strip():
            return "⚠️ Could not extract meaningful content from the page."

        # Summarize with AI
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "Summarize the webpage content in 3–4 concise sentences."},
                {"role": "user", "content": text_content}
            ],
            max_tokens=150
        )
        return completion.choices[0].message.content

    except requests.exceptions.RequestException as e:
        return f"⚠️ Failed to fetch URL: {str(e)}"
    except Exception as e:
        return f"⚠️ Error summarizing URL: {str(e)}"


# def summarize_url(url):
#     """Placeholder function to summarize a URL's content."""
#     if not url or not url.startswith(("http://", "https://")):
#         return "Please enter a valid URL."

    # You would add your web scraping and summarization logic here.
    return f"This is a placeholder summary of the content from the URL: {url}"

# -------------------------------------------------------------------------------------

# ----------------------------------------------------------
# ----------------- Gradio App Logic -----------------
# ----------------------------------------------------------


def respond(user_input, audio_filepath, history, url_input=None, file_input=None):
    global voice_enabled

    # Handle summarization requests first
    if url_input:
        summary = summarize_url(url_input)
        history.append((f"Summarize URL: {url_input}", summary))
        return history, "", None, None

    if file_input:
        summary = summarize_file(file_input)
        history.append((f"Summarize File: {file_input.name}", summary))
        return history, "", None, None

    # Handle chat messages
    if not user_input and not audio_filepath:
        return history, "", None, None

    message = user_input
    if audio_filepath:
        # NOTE: You need to implement your own transcription logic here.
        message = "Transcribing audio... (This feature needs a real transcription tool)"

    # Handle voice toggle commands
    if message.lower() == "voice on":
        voice_enabled = True
        response = "🔊 Voice enabled."
    elif message.lower() == "voice off":
        voice_enabled = False
        response = "🔇 Voice disabled."
    else:
        try:
            response = chat_with_agent(message, history)
        except Exception as e:
            tb = traceback.format_exc()
            logging.exception("Error while calling agent")
            response = f"⚠️ Agent Error: {str(e)}\n\n```\n{tb}\n```"

    audio_output = None
    if voice_enabled and not response.startswith("⚠️"):
        try:
            audio_output_file = text_to_speech(response)
            if audio_output_file and os.path.exists(audio_output_file):
                audio_output = audio_output_file
        except Exception as e:
            logging.exception("TTS generation failed")
            audio_output = None

    if audio_output:
        history.append((message, (response, audio_output)))
    else:
        history.append((message, response))

    return history, "", None, None

# -------------------------------------------------------------------------------------


def respond_with_tts(user_text, mic_file, history, url_input, file_upload):
    if history is None:
        history = []

    # 1️⃣ Append user message immediately
    history.append((user_text, ""))

    # 2️⃣ Generate bot response
    bot_response = respond(user_text, mic_file, history, url_input, file_upload)[0][-1][1]

    # 3️⃣ Update last entry with bot response
    history[1] = (user_text, bot_response)

    # 4️⃣ Convert bot response to speech
    audio_file = tts_tool.run(bot_response)

    return history, None, None, None, audio_file



# ------------------------------------------------------------------
# ----------------- Gradio UI -----------------
# ------------------------------------------------------------------

with gr.Blocks(css="""
    .gr-chat-message.user {background-color: #E0F7FA; border-radius: 10px; padding: 5px;}
    .gr-chat-message.bot {background-color: #FFF3E0; border-radius: 10px; padding: 5px;}
    .gr-button {background-color: #29B6F6; color: white;}
""") as demo:

    gr.Markdown("## 🤖 AI Content Coach")
    gr.Markdown("Type your question or speak it. Use the sidebar for additional features.")

    chatbot = gr.Chatbot(label="Chat History", elem_id="chatbot", type="tuples", height=400)
    history = gr.State([])   # ✅ persistent chat history

    # Audio output for TTS
    audio_output = gr.Audio(label="Speech Output", type="filepath")

    with gr.Row():
        with gr.Column(scale=3):
            msg = gr.Textbox(label="Your Question", placeholder="Type here...", lines=1)
            mic = gr.Microphone(label="🎤 Speak", type="filepath", sources="microphone")
            clear_button = gr.Button("Clear Chat", variant="primary")

        with gr.Column(scale=1):
            gr.Markdown("### Features")
            url_input = gr.Textbox(label="Summarize URL", placeholder="Paste a URL here")
            file_upload = gr.File(label="Summarize File", file_types=["pdf", "txt"])
            download_button = gr.Button("Download Chat Transcript")
            file_output = gr.File(label="Download File")

    # ----------------- Event listeners -----------------
    msg.submit(
        fn=respond_with_tts,
        inputs=[msg, gr.State(None), history, url_input, file_upload],
        outputs=[chatbot, msg, url_input, file_upload, audio_output]
    )

    mic.change(
        fn=respond_with_tts,
        inputs=[gr.State(None), mic, history, url_input, file_upload],
        outputs=[chatbot, msg, url_input, file_upload, audio_output]
    )

    url_input.submit(
        fn=respond_with_tts,
        inputs=[gr.State(None), gr.State(None), history, url_input, file_upload],
        outputs=[chatbot, msg, url_input, file_upload, audio_output]
    )

    file_upload.change(
        fn=respond_with_tts,
        inputs=[gr.State(None), gr.State(None), history, url_input, file_upload],
        outputs=[chatbot, msg, url_input, file_upload, audio_output]
    )

    clear_button.click(lambda: ([], None, None, None, None), inputs=None, outputs=[chatbot, msg, url_input, file_upload, audio_output])

    download_button.click(
        fn=download_transcript,
        inputs=[history],
        outputs=[file_output]
    )

# Launch the app
demo.launch()


/tmp/ipython-input-43636367.py:306: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Chat History", elem_id="chatbot", type="tuples", height=400)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1d2bf2e04446517f7f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
